In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets

import glob
import zipfile

In [2]:
%ls

Data.zip  Solution.ipynb


In [3]:
files = glob.glob('*.zip')
files

['Data.zip']

In [4]:
for file in files:
    print('Unzipping:',file)

    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('')

Unzipping: Data.zip


In [5]:
# Set the paths to the train and test data folders
train_data_path = 'Data/train'
test_data_path = 'Data/test'

In [6]:
# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
# Load the datasets
train_dataset = datasets.ImageFolder(train_data_path, transform=transform)
test_dataset = datasets.ImageFolder(test_data_path, transform=transform)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(device))
print(torch.cuda.get_device_properties(device))

NVIDIA GeForce RTX 3090
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3090', major=8, minor=6, total_memory=24259MB, multi_processor_count=82)


In [9]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 56 * 56, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [10]:
# Create an instance of the CNN model
model = CNN(num_classes=len(train_dataset.classes)).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {loss.item():.4f}")

Epoch [1/10] - Loss: 0.6870
Epoch [2/10] - Loss: 0.6970
Epoch [3/10] - Loss: 0.6789
Epoch [4/10] - Loss: 0.6752
Epoch [5/10] - Loss: 0.6981
Epoch [6/10] - Loss: 0.6993
Epoch [7/10] - Loss: 0.6601
Epoch [8/10] - Loss: 0.7432
Epoch [9/10] - Loss: 0.7390
Epoch [10/10] - Loss: 0.7328


In [12]:
# Evaluate the model on the test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()

print(f"Test Accuracy: {(correct/total)*100:.2f}%")

Test Accuracy: 55.67%


In [20]:
torch.save(model.state_dict(), 'cnn.pth')

In [21]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(model.state_dict())

OrderedDict([   (   'features.0.weight',
                    tensor([[[[-9.6221e-02,  7.6613e-02,  8.1389e-02],
          [ 1.1529e-01, -1.0570e-01, -1.4914e-02],
          [-6.7721e-02,  6.8485e-02, -5.6099e-02]],

         [[-8.2438e-02, -1.4025e-02, -2.0519e-02],
          [ 1.4752e-02,  1.4239e-01,  2.4689e-01],
          [-6.2407e-02, -1.1078e-01, -6.0933e-02]],

         [[ 5.6897e-02, -2.7823e-02, -9.8780e-02],
          [ 2.1554e-01, -7.0730e-02,  3.7382e-02],
          [ 5.5116e-02,  1.4232e-01,  4.9285e-02]]],


        [[[ 5.0445e-02, -5.3650e-02,  1.2804e-02],
          [-1.3300e-01, -5.0701e-02,  8.9329e-03],
          [ 1.7965e-02, -5.6841e-02, -1.4984e-01]],

         [[ 9.3845e-02, -6.0375e-03, -1.1849e-01],
          [ 1.9553e-02, -1.1881e-01, -1.9762e-01],
          [ 9.5614e-02,  9.9842e-02, -1.0016e-01]],

         [[ 3.0299e-02,  9.2972e-02,  7.9159e-02],
          [-1.1616e-01, -1.7750e-02, -7.6257e-02],
          [-7.9690e-02, -5.2625e-02, -1.5719e-01]]],


     

In [24]:
# Create new model and load states
newmodel = CNN().to(device)
newmodel.load_state_dict(torch.load("cnn.pth"))

n_pp = pprint.PrettyPrinter(indent=4)
n_pp.pprint(newmodel.state_dict())

OrderedDict([   (   'features.0.weight',
                    tensor([[[[-9.6221e-02,  7.6613e-02,  8.1389e-02],
          [ 1.1529e-01, -1.0570e-01, -1.4914e-02],
          [-6.7721e-02,  6.8485e-02, -5.6099e-02]],

         [[-8.2438e-02, -1.4025e-02, -2.0519e-02],
          [ 1.4752e-02,  1.4239e-01,  2.4689e-01],
          [-6.2407e-02, -1.1078e-01, -6.0933e-02]],

         [[ 5.6897e-02, -2.7823e-02, -9.8780e-02],
          [ 2.1554e-01, -7.0730e-02,  3.7382e-02],
          [ 5.5116e-02,  1.4232e-01,  4.9285e-02]]],


        [[[ 5.0445e-02, -5.3650e-02,  1.2804e-02],
          [-1.3300e-01, -5.0701e-02,  8.9329e-03],
          [ 1.7965e-02, -5.6841e-02, -1.4984e-01]],

         [[ 9.3845e-02, -6.0375e-03, -1.1849e-01],
          [ 1.9553e-02, -1.1881e-01, -1.9762e-01],
          [ 9.5614e-02,  9.9842e-02, -1.0016e-01]],

         [[ 3.0299e-02,  9.2972e-02,  7.9159e-02],
          [-1.1616e-01, -1.7750e-02, -7.6257e-02],
          [-7.9690e-02, -5.2625e-02, -1.5719e-01]]],


     

In [ ]:
# Evaluate the model on the test data
newmodel.eval()
n_correct = 0
n_total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = newmodel(inputs.to(device))
        _, predicted = torch.max(outputs.data, 1)
        n_total += labels.size(0)
        n_correct += (predicted == labels.to(device)).sum().item()

print(f"Loaded Model Test Accuracy: {(correct/total)*100:.2f}%")